Steane code
==========

Sets up logical zero for Steane code based  on the parity matrix in the book by Nielsen MA, Chuang IL. Quantum Computation and Quantum Information, 10th Anniversary Edition. Cambridge University Press; 2016. p. 474

The necessary function modules are imported , including the SteaneCodeLogicalQubit class.  The methods of this class are called in this notebook.

In [1]:
from qiskit import(
  QuantumCircuit,
  QuantumRegister,
  ClassicalRegister,
  execute,
  Aer)

from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import pauli_error, depolarizing_error
from qiskit.compiler import transpile

from circuits import SteaneCodeLogicalQubit

from helper_functions import (
    count_valid_output_strings,
    find_individual_ancilla_values,
    find_ancilla_values,
    find_parity,
    get_noise,
    mean_of_list,
    calculate_standard_error
    )

Define constants so the process flow can be controlled from one place:

In [2]:
NOISE = True                             #Test with noise
SHOTS = 10000                            #Number of shots to run    
MEASURE_NOISE = 0.0046                   #Measurement noise not relevant
SINGLE_GATE_DEPOLARISING = 0.000366      #Single gate noise   
TWO_GATE_DEPOLARISING = 0.0204           #Two gate noise
ITERATIONS = 25

SIMULATOR = Aer.get_backend('qasm_simulator')

We specify the parity check matrix, since this defines the Steane code.  It is validated before the logical qubit is initiated to check that it is orthogonal to the valid codewords.

In [3]:
parity_check_matrix = [[0,0,0,1,1,1,1],
                       [0,1,1,0,0,1,1],
                       [1,0,1,0,1,0,1]]

The allowed codewords for the zero logical from Nielsen and Chuang is stored in a list.  Note, the order of qubits in the list is reversed compared to the normal output of Qiskit.

In [4]:
codewords = [[0,0,0,0,0,0,0],   
             [1,0,1,0,1,0,1],
             [0,1,1,0,0,1,1],
             [1,1,0,0,1,1,0],
             [0,0,0,1,1,1,1],
             [1,0,1,1,0,1,0],
             [0,1,1,1,1,0,0],
             [1,1,0,1,0,0,1]]

Encoding set up
-----------------------
The encoding circuit alone is set up and either transpiled to the native gate set for the Ion computer, or calculated with the standard IBM noise set


In [6]:
print(f'Run with {SHOTS} shots and {ITERATIONS} iterations')
for ion_gate_set in [True, False]:
    results_list = []
    if ion_gate_set:
        print('Results after transpilation to ion computer gate set')   
    else:
        print('Results without transpilation')
    for iteration in range(ITERATIONS):
        qubit = SteaneCodeLogicalQubit(1, parity_check_matrix, codewords)
        qubit.set_up_logical_zero()
        qubit.barrier()
        if ion_gate_set:
            SINGLE_GATE_SET = ['id', 'ry', 'rx']
            TWO_GATE_SET = ['rxx']
        else:
            SINGLE_GATE_SET = ['id', 'h', 'x', 'y', 'z,']
            TWO_GATE_SET = ['cx', 'cz']    
        BASIS_GATE_SET = SINGLE_GATE_SET + TWO_GATE_SET
        if NOISE:
            noise_model = get_noise(MEASURE_NOISE, SINGLE_GATE_DEPOLARISING, TWO_GATE_DEPOLARISING, 
                                   SINGLE_GATE_SET, TWO_GATE_SET)
        #print('noise_model',noise_model)
        qubit.logical_measure()
        qubit.barrier()
        if ion_gate_set:
            qt = transpile(qubit, basis_gates = BASIS_GATE_SET)
            if NOISE:
                result = execute(qt, SIMULATOR, noise_model = noise_model, shots = SHOTS).result()
            else:
                result = execute(qt, SIMULATOR, shots = SHOTS).result()
            counts = result.get_counts(qt)
        else:
            if NOISE:
                result = execute(qubit, SIMULATOR, noise_model = noise_model, shots = SHOTS).result()
            else:
                result = execute(qubit, SIMULATOR, shots = SHOTS).result()
            counts = result.get_counts(qubit)
        count_valid, count_invalid = count_valid_output_strings(counts, codewords, 2)
        invalid_rate = count_invalid / SHOTS
        results_list.append(invalid_rate)
        print(f'There were {count_valid} valid output strings and {count_invalid} invalid output string')
        print(f'Invalid strings occurred on {invalid_rate:.4f} of runs')
    mean = mean_of_list(results_list)
    standard_deviation, standard_error = calculate_standard_error(results_list)
    print()
    print(f' Over all iterations invalid strings occurred on {mean:.4f} of runs')
    print(f' The standard deviation {standard_deviation:.4f} and the standard error is {standard_error:.4f}')
    print()

Run with 10000 shots and 25 iterations
Results after transpilation to ion computer gate set
There were 8076 valid output strings and 1924 invalid output string
Invalid strings occurred on 0.1924 of runs
There were 8013 valid output strings and 1987 invalid output string
Invalid strings occurred on 0.1987 of runs
There were 8065 valid output strings and 1935 invalid output string
Invalid strings occurred on 0.1935 of runs
There were 8054 valid output strings and 1946 invalid output string
Invalid strings occurred on 0.1946 of runs
There were 8055 valid output strings and 1945 invalid output string
Invalid strings occurred on 0.1945 of runs
There were 8085 valid output strings and 1915 invalid output string
Invalid strings occurred on 0.1915 of runs
There were 8009 valid output strings and 1991 invalid output string
Invalid strings occurred on 0.1991 of runs
There were 8045 valid output strings and 1955 invalid output string
Invalid strings occurred on 0.1955 of runs
There were 8004 vali

Draw circuit:

In [ ]:
qubit.draw(output = 'mpl', filename = 'Steane_code_encoding_circuit.jpg', fold = 30)

Draw transpiled circuit

In [ ]:
qt.draw(output = 'mpl', filename = 'Steane_code_encoding_circuit_transpiled.jpg', fold = 40)